# 수능 국어 문제 풀이

# 라이브러리 import 및 설정
필요한 라이브러리들을 임포트하고 OpenAI 클라이언트를 초기화합니다. 시간 측정을 위한 전역 변수들도 함께 설정하여 후속 분석에서 각 조합별 성능과 처리 시간을 추적할 수 있도록 준비합니다.

In [17]:
from openai import OpenAI  # 새로운 방식
import pandas as pd
import json
import re
import time
from typing import List, Dict, Any, Tuple, Optional
import os
from datetime import datetime, timedelta

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")  # 실제 API 키로 변경하거나 환경변수 OPENAI_API_KEY 사용
)


# 시간 측정을 위한 전역 변수들
각 문제들을 푸는 데 걸리는 시간을 재기 위한 변수들을 정의합니다.

In [18]:
timing_results = {}  # 각 조합별 시간 측정 결과
paragraph_timings = {}  # 단락별 시간 측정
question_timings = {}  # 문제별 시간 측정

# 데이터 로드 및 전처리
수능 국어 문제 데이터를 JSON 파일에서 로드하는 함수를 정의합니다. 샘플 데이터를 포함하여 실제 문제 세트의 구조를 확인할 수 있으며, 전체 문제 세트의 개수를 출력하여 데이터 로딩이 성공적으로 완료되었는지 확인합니다.

In [19]:
# JSON 파일에서 문제 데이터 로드
def load_problems(file_path: str) -> List[Dict]:
    """문제 데이터를 JSON 파일에서 로드"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return []
    except json.JSONDecodeError:
        print("JSON 파일 형식이 올바르지 않습니다.")
        return []

all_problems = load_problems('2023_11_KICE.json')

print(f"로드된 문제 세트 수: {len(all_problems)}")

로드된 문제 세트 수: 11


# 모델 및 프롬프트 정의
실험에 사용할 GPT 모델들과 다양한 프롬프트 템플릿을 정의합니다.
zero_shot, negative, function_calling 방식의 프롬프트를 준비하고, function_calling의 경우 문제 유형별로 세분화된 프롬프트를 제공하여 보다 정교한 문제 해결이 가능하도록 구성합니다.
## 문제의 유형
- question_plus : 보기 지문이 있는 문제
- vocabulary : 문법과 어휘에 대한 문
- comprehension : 지문의 이해에 대한 문제
- inference : 지문을 바탕으로 추론하는 문제

In [20]:
# 사용할 모델들
models = ["gpt-4o-mini", "gpt-4o"]

# 프롬프트 템플릿들
prompts = {
    "zero_shot": """다음 지문을 읽고 문제를 풀어주세요.

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

    "negative": """다음 지문을 읽고 문제를 풀어주세요.

주의사항:
- 추측하지 말고 지문의 내용에 근거하여 답해주세요
- 확실하지 않은 답은 피해주세요
- 선택지를 꼼꼼히 비교 분석해주세요

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

    "function_calling": {
        "question_plus": """다음은 추가 지문이 있는 문제입니다. 본문과 추가 지문을 모두 참고하여 답해주세요.

지문:
{paragraph}

문제: {question}

추가 지문:
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

        "vocabulary": """다음은 어휘 및 문법 문제입니다. 정확한 어휘의 의미와 문법 규칙을 적용하여 답해주세요.

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

        "comprehension": """다음은 내용 파악 문제입니다. 지문의 내용을 정확히 이해하고 답해주세요.

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

        "inference": """다음은 추론 문제입니다. 지문의 내용을 바탕으로 논리적으로 추론하여 답해주세요.

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나)."""
    }
}

In [21]:

# 전역 변수 (현재 사용 중인 모델과 프롬프트)
current_model = None
current_prompt_type = None

print("모델 및 프롬프트 설정 완료")

모델 및 프롬프트 설정 완료



# 유틸리티 함수들
시간 측정과 관련된 유틸리티 함수들을 정의합니다. 초 단위의 시간을 읽기 쉬운 형태로 포맷팅하는 함수와 TimingTracker 클래스를 통해 정확한 시간 측정이 가능하도록 구성하여 성능 분석의 정확성을 높입니다.

In [22]:
def format_time_duration(seconds: float) -> str:
    """초를 읽기 쉬운 형태로 포맷팅"""
    if seconds < 1:
        return f"{seconds*1000:.0f}ms"
    elif seconds < 60:
        return f"{seconds:.2f}s"
    else:
        minutes = int(seconds // 60)
        remaining_seconds = seconds % 60
        return f"{minutes}m {remaining_seconds:.2f}s"


def get_current_timestamp() -> str:
    """현재 시간을 문자열로 반환"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [23]:
class TimingTracker:
    """시간 측정을 위한 클래스"""
    
    def __init__(self):
        self.start_time = None
        self.end_time = None
        self.duration = None
    
    def start(self):
        """시간 측정 시작"""
        self.start_time = time.time()
        return self
    
    def stop(self):
        """시간 측정 종료"""
        self.end_time = time.time()
        self.duration = self.end_time - self.start_time
        return self.duration
    
    def get_duration(self) -> float:
        """측정된 시간 반환"""
        return self.duration if self.duration else 0

functional calling을 위해 문제의 유형을 구분하는 과정이 있는 함수입니다.
이 과정에서 api를 추가 사용하며, gpt-4o-mini 모델을 사용하여 분류했습니다.

In [ ]:
def get_question_type_by_model(question: str, question_plus: Optional[str] = None) -> Tuple[str, float]:
    """GPT 모델을 통해 문제 유형을 분류 (시간 측정 포함)"""
    
    timer = TimingTracker().start()
    
    classification_prompt = f"""다음 문제의 유형을 분류해주세요.

문제: {question}
추가내용: {question_plus if question_plus else "없음"}

다음 중 하나로 분류해주세요:
1. question_plus - 추가 지문이나 자료가 있는 문제
2. vocabulary - 어휘나 문법 관련 문제  
3. comprehension - 내용 파악 문제
4. inference - 내용 기반 추론 문제

분류 결과만 답해주세요 (question_plus, vocabulary, comprehension, inference 중 하나):"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": classification_prompt}],
            max_tokens=10,
            temperature=0
        )
        
        classification_time = timer.stop()
        result = response.choices[0].message.content.strip().lower()
        
        # 결과 검증 및 기본값 설정
        valid_types = ["question_plus", "vocabulary", "comprehension", "inference"]
        for valid_type in valid_types:
            if valid_type in result:
                return valid_type, classification_time
        
        # 기본값: question_plus가 있으면 question_plus, 없으면 comprehension
        return ("question_plus" if question_plus else "comprehension"), classification_time
        
    except Exception as e:
        classification_time = timer.stop()
        print(f"문제 유형 분류 중 오류 발생: {e}")
        return ("question_plus" if question_plus else "comprehension"), classification_time



유틸리티 함수 정의 완료


모델의 응답에서 정답 번호만 추출하는 함수입니다.

In [ ]:

def extract_answer_from_response(response: str, choices: List[str]) -> int:
    """모델 응답에서 정답 번호를 추출"""
    
    # 1~5 숫자 패턴 찾기
    number_patterns = [
        r'(?:정답은?\s*)?([1-5])(?:번)?',
        r'([1-5])',
        r'(?:답|선택):\s*([1-5])',
    ]
    
    for pattern in number_patterns:
        matches = re.findall(pattern, response)
        if matches:
            try:
                answer_num = int(matches[0])
                if 1 <= answer_num <= 5:
                    return answer_num
            except ValueError:
                continue
    
    # 선택지 텍스트 매칭 시도
    for i, choice in enumerate(choices):
        # 선택지의 주요 키워드가 응답에 포함되어 있는지 확인
        choice_keywords = choice.split()[:3]  # 처음 3단어만 사용
        if len(choice_keywords) >= 2:
            keyword_phrase = ' '.join(choice_keywords)
            if keyword_phrase in response:
                return i + 1
    
    # 기본값 (추출 실패 시)
    print(f"답안 추출 실패. 응답: {response[:100]}...")
    return 1


def format_choices(choices: List[str]) -> str:
    """선택지를 문자열로 포맷팅"""
    formatted = []
    for i, choice in enumerate(choices, 1):
        formatted.append(f"{i}. {choice}")
    return '\n'.join(formatted)


def call_openai_api(prompt: str, model: str, max_retries: int = 3) -> Tuple[str, float]:
    """OpenAI API 호출 (재시도 로직 포함) - 새로운 방식, 시간 측정 포함"""
    
    timer = TimingTracker().start()
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=0
            )
            api_time = timer.stop()
            return response.choices[0].message.content.strip(), api_time
            
        except Exception as e:
            print(f"API 호출 실패 (시도 {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # 지수 백오프
            else:
                api_time = timer.stop()
                return "API 호출 실패", api_time

print("유틸리티 함수 정의 완료")


# prediction 함수
주어진 문제들에 대해 현재 설정된 모델과 프롬프트로 예측 수행 (시간 측정 포함)
    
Args:
  problems: 문제 리스트 (각 문제는 question, choices, answer, score 포함)
  problem_set_id: 문제 세트 ID
    
Returns:
  Tuple[List[int], Dict]: 각 문제에 대한 예측 답안 번호와 시간 측정 결과

In [25]:
def prediction(problems: List[Dict], problem_set_id: str) -> Tuple[List[int], Dict]:

    global current_model, current_prompt_type
    
    if not current_model or not current_prompt_type:
        raise ValueError("current_model과 current_prompt_type이 설정되지 않았습니다.")
    
    predictions = []
    question_times = []
    classification_times = []
    api_times = []
    
    for i, problem in enumerate(problems):
        question_timer = TimingTracker().start()
        
        question = problem['question']
        choices = problem['choices']
        question_plus = problem.get('question_plus', '')
        
        # 프롬프트 선택 및 구성
        classification_time = 0
        if current_prompt_type == "function_calling":
            # 모델이 문제 유형을 판단
            question_type, classification_time = get_question_type_by_model(question, question_plus)
            prompt_template = prompts[current_prompt_type][question_type]
            print(f"  문제 {i+1}: Function Calling - {question_type} 타입 (분류 시간: {format_time_duration(classification_time)})")
        else:
            prompt_template = prompts[current_prompt_type]
        
        # 프롬프트 포맷팅
        formatted_prompt = prompt_template.format(
            paragraph=current_paragraph,  # 전역 변수에서 참조
            question=question,
            question_plus=f"\n추가 내용:\n{question_plus}" if question_plus else "",
            choices=format_choices(choices)
        )
        
        # API 호출
        response, api_time = call_openai_api(formatted_prompt, current_model)
        
        # 답안 추출
        predicted_answer = extract_answer_from_response(response, choices)
        predictions.append(predicted_answer)
        
        # 시간 측정 완료
        total_question_time = question_timer.stop()
        
        # 시간 데이터 저장
        question_times.append(total_question_time)
        classification_times.append(classification_time)
        api_times.append(api_time)
        
        print(f"  문제 {i+1} 응답: {response[:50]}... → 예측: {predicted_answer} (시간: {format_time_duration(total_question_time)})")
    
    # 시간 측정 결과 정리
    timing_data = {
        'problem_set_id': problem_set_id,
        'total_questions': len(problems),
        'question_times': question_times,
        'classification_times': classification_times,
        'api_times': api_times,
        'avg_question_time': sum(question_times) / len(question_times),
        'total_question_time': sum(question_times),
        'avg_api_time': sum(api_times) / len(api_times),
        'total_classification_time': sum(classification_times)
    }
    
    return predictions, timing_data

print("prediction 함수 정의 완료")

prediction 함수 정의 완료


# 정답 및 배점 데이터 준비

In [26]:
# 수능 정답 저장 배열 생성
correct_answers = []
scores = []
problem_info = []  # 문제 정보 저장 (디버깅용)

for problem_set in all_problems:
    for problem in problem_set['problems']:
        correct_answers.append(problem['answer'])
        scores.append(problem['score'])
        problem_info.append({
            'id': problem_set['id'],
            'question': problem['question'][:50] + "..." if len(problem['question']) > 50 else problem['question']
        })

print(f"총 문제 수: {len(correct_answers)}")
print(f"총 배점: {sum(scores)}점")

총 문제 수: 45
총 배점: 100점


# 프롬프트, 모델 조합 정의 및 실행 준비

In [27]:
# 각 프롬프트별 조합 정의
combinations = [
    ("gpt-4o-mini", "zero_shot"),
    ("gpt-4o-mini", "negative"), 
    ("gpt-4o-mini", "function_calling"),
    ("gpt-4o", "zero_shot"),
    ("gpt-4o", "negative"),
    ("gpt-4o", "function_calling")
]

# 조합별 점수 저장 딕셔너리
combination_scores = {}
detailed_results = {}  # 상세 결과 저장

print(f"실행할 조합 수: {len(combinations)}")
for i, (model, prompt) in enumerate(combinations, 1):
    print(f"{i}. {model} + {prompt}")


실행할 조합 수: 6
1. gpt-4o-mini + zero_shot
2. gpt-4o-mini + negative
3. gpt-4o-mini + function_calling
4. gpt-4o + zero_shot
5. gpt-4o + negative
6. gpt-4o + function_calling


문제를 푸는 함수입니다.
각 문제를 풀고 나면 걸리는 시간과 정답과 오답을 비교합니다.

In [ ]:

def solve_problem(all_problems: List[Dict], combination: Tuple[str, str]) -> Tuple[int, Dict]:
    """
    주어진 문제들을 특정 모델-프롬프트 조합으로 해결 (시간 측정 포함)
    
    Returns:
        Tuple[int, Dict]: 총 획득 점수와 시간 측정 결과
    """
    global current_model, current_prompt_type, current_paragraph
    
    model, prompt_type = combination
    current_model, current_prompt_type = model, prompt_type
    combination_key = f"{model}_{prompt_type}"
    
    print(f"\n{'='*60}")
    print(f"실행 중: {model} + {prompt_type}")
    print(f"시작 시간: {get_current_timestamp()}")
    print(f"{'='*60}")
    
    # 전체 실행 시간 측정
    total_timer = TimingTracker().start()
    
    total_score = 0
    problem_idx = 0
    detailed_result = []
    paragraph_timing_data = []
    all_question_timings = []
    
    for problem_set in all_problems:
        paragraph_timer = TimingTracker().start()
        
        current_paragraph = problem_set['paragraph']
        problems = problem_set['problems']
        
        print(f"\n문제 세트 ID: {problem_set['id']}")
        print(f"단락 길이: {len(current_paragraph)}자")
        print("-" * 40)
        
        # 모델 예측 수행 (시간 측정 포함)
        model_predictions, question_timing_data = prediction(problems, problem_set['id'])
        
        # 단락 처리 시간 측정 완료
        paragraph_time = paragraph_timer.stop()
        
        # 각 문제별 정답 비교
        for i, (pred, problem) in enumerate(zip(model_predictions, problems)):
            correct = problem['answer']
            score = problem['score']
            
            is_correct = (pred == correct)
            earned_score = score if is_correct else 0
            total_score += earned_score
            
            result_symbol = "✓" if is_correct else "✗"
            question_time = question_timing_data['question_times'][i]
            print(f"  {result_symbol} 문제 {i+1}: 예측={pred}, 정답={correct}, 점수={earned_score}/{score} (시간: {format_time_duration(question_time)})")
            
            # 상세 결과 저장
            detailed_result.append({
                'problem_set_id': problem_set['id'],
                'problem_num': i + 1,
                'question': problem['question'][:100] + "..." if len(problem['question']) > 100 else problem['question'],
                'predicted': pred,
                'correct': correct,
                'is_correct': is_correct,
                'score': earned_score,
                'max_score': score,
                'question_time': question_time,
                'api_time': question_timing_data['api_times'][i],
                'classification_time': question_timing_data['classification_times'][i]
            })
            
            problem_idx += 1
        
        # 단락별 시간 데이터 저장
        paragraph_timing_data.append({
            'problem_set_id': problem_set['id'],
            'paragraph_length': len(current_paragraph),
            'num_questions': len(problems),
            'paragraph_time': paragraph_time,
            'avg_question_time': question_timing_data['avg_question_time'],
            'total_api_time': sum(question_timing_data['api_times']),
            'total_classification_time': question_timing_data['total_classification_time']
        })
        
        all_question_timings.extend(question_timing_data['question_times'])
        
        print(f"단락 처리 시간: {format_time_duration(paragraph_time)} (문제당 평균: {format_time_duration(question_timing_data['avg_question_time'])})")
    
    # 전체 실행 시간 완료
    total_execution_time = total_timer.stop()
    
    accuracy = sum(1 for r in detailed_result if r['is_correct']) / len(detailed_result) * 100
    max_possible_score = sum(scores)
    
    print(f"\n{'='*60}")
    print(f"최종 결과: {model} + {prompt_type}")
    print(f"완료 시간: {get_current_timestamp()}")
    print(f"총 실행 시간: {format_time_duration(total_execution_time)}")
    print(f"총 점수: {total_score}/{max_possible_score}점 ({total_score/max_possible_score*100:.1f}%)")
    print(f"정답률: {accuracy:.1f}% ({sum(1 for r in detailed_result if r['is_correct'])}/{len(detailed_result)})")
    print(f"{'='*60}")
    
    # 시간 분석 결과
    timing_summary = {
        'combination': combination_key,
        'total_execution_time': total_execution_time,
        'total_paragraphs': len(all_problems),
        'total_questions': len(detailed_result),
        'avg_paragraph_time': sum(p['paragraph_time'] for p in paragraph_timing_data) / len(paragraph_timing_data),
        'avg_question_time': sum(all_question_timings) / len(all_question_timings),
        'total_api_time': sum(r['api_time'] for r in detailed_result),
        'total_classification_time': sum(r['classification_time'] for r in detailed_result),
        'paragraph_timings': paragraph_timing_data,
        'question_timings': [r['question_time'] for r in detailed_result]
    }
    
    print(f"\n⏱️  시간 분석:")
    print(f"   단락당 평균 시간: {format_time_duration(timing_summary['avg_paragraph_time'])}")
    print(f"   문제당 평균 시간: {format_time_duration(timing_summary['avg_question_time'])}")
    print(f"   전체 API 호출 시간: {format_time_duration(timing_summary['total_api_time'])}")
    if timing_summary['total_classification_time'] > 0:
        print(f"   전체 분류 시간: {format_time_duration(timing_summary['total_classification_time'])}")
    
    # 결과 저장
    combination_scores[combination_key] = total_score
    detailed_results[combination_key] = detailed_result
    timing_results[combination_key] = timing_summary
    
    return total_score, timing_summary

print("solve_problem 함수 정의 완료")


In [29]:

# ==================== Cell 10: 메인 실행 루프 ====================


In [38]:
print("🚀 실험 시작!")
print(f"총 {len(combinations)}개 조합을 실행합니다.")
print(f"실험 시작 시간: {get_current_timestamp()}")

experiment_start_time = time.time()

# 각 조합별로 실행
for i, combination in enumerate(combinations, 1):
    print(f"\n🔄 진행률: {i}/{len(combinations)}")
    solve_problem(all_problems, combination)
    
    # API 호출 제한을 고려한 대기 시간
    if i < len(combinations):
        print("⏳ 다음 조합 실행을 위해 잠시 대기...\n\n")
        time.sleep(2)

experiment_end_time = time.time()
total_experiment_time = experiment_end_time - experiment_start_time

print(f"\n🎉 모든 실험 완료!")
print(f"전체 실험 시간: {format_time_duration(total_experiment_time)}")
print(f"실험 종료 시간: {get_current_timestamp()}")


🚀 실험 시작!
총 6개 조합을 실행합니다.
실험 시작 시간: 2025-05-23 16:33:32

🔄 진행률: 1/6

실행 중: gpt-4o-mini + zero_shot
시작 시간: 2025-05-23 16:33:32

문제 세트 ID: 2023_11_KICE_1-3
단락 길이: 978자
----------------------------------------
  문제 1 응답: 4... → 예측: 4 (시간: 1.14s)
  문제 2 응답: 4... → 예측: 4 (시간: 830ms)
  문제 3 응답: 1... → 예측: 1 (시간: 774ms)
  ✓ 문제 1: 예측=4, 정답=4, 점수=2/2 (시간: 1.14s)
  ✗ 문제 2: 예측=4, 정답=5, 점수=0/3 (시간: 830ms)
  ✓ 문제 3: 예측=1, 정답=1, 점수=2/2 (시간: 774ms)
단락 처리 시간: 2.74s (문제당 평균: 914ms)

문제 세트 ID: 2023_11_KICE_4-9
단락 길이: 2158자
----------------------------------------
  문제 1 응답: 4... → 예측: 4 (시간: 817ms)
  문제 2 응답: 5... → 예측: 5 (시간: 660ms)
  문제 3 응답: 3... → 예측: 3 (시간: 979ms)
  문제 4 응답: 2... → 예측: 2 (시간: 818ms)
  문제 5 응답: 3... → 예측: 3 (시간: 820ms)
  문제 6 응답: 2.... → 예측: 2 (시간: 818ms)
  ✓ 문제 1: 예측=4, 정답=4, 점수=2/2 (시간: 817ms)
  ✓ 문제 2: 예측=5, 정답=5, 점수=2/2 (시간: 660ms)
  ✓ 문제 3: 예측=3, 정답=3, 점수=2/2 (시간: 979ms)
  ✓ 문제 4: 예측=2, 정답=2, 점수=2/2 (시간: 818ms)
  ✗ 문제 5: 예측=3, 정답=5, 점수=0/3 (시간: 820ms)
  ✓ 문제 6: 예측=2, 정답=2, 점수=2/

# 결과 분석
## 테이블 형태로 정리하는 함수

In [39]:
def create_results_table(combination_scores: Dict[str, int], timing_results: Dict[str, Dict]) -> pd.DataFrame:
    """결과를 테이블 형태로 정리 (시간 정보 포함)"""
    
    # 데이터 재구성
    data = []
    for combination_key, score in combination_scores.items():
        model, prompt = combination_key.split('_', 1)
        timing_data = timing_results.get(combination_key, {})
        
        data.append({
            'Model': model,
            'Prompt': prompt,
            'Score': score,
            'Max_Score': sum(scores),
            'Accuracy_%': round(score / sum(scores) * 100, 1),
            'Total_Time_s': round(timing_data.get('total_execution_time', 0), 2),
            'Avg_Question_Time_s': round(timing_data.get('avg_question_time', 0), 3),
            'Avg_Paragraph_Time_s': round(timing_data.get('avg_paragraph_time', 0), 2),
            'Questions_Per_Minute': round(60 / timing_data.get('avg_question_time', 1), 1) if timing_data.get('avg_question_time', 0) > 0 else 0
        })
    
    df = pd.DataFrame(data)
    return df

def create_timing_analysis_table(timing_results: Dict[str, Dict]) -> pd.DataFrame:
    """시간 분석 전용 테이블 생성"""
    
    data = []
    for combination_key, timing_data in timing_results.items():
        model, prompt = combination_key.split('_', 1)
        
        data.append({
            'Combination': combination_key,
            'Model': model,
            'Prompt': prompt,
            'Total_Time': format_time_duration(timing_data.get('total_execution_time', 0)),
            'Avg_Question_Time': format_time_duration(timing_data.get('avg_question_time', 0)),
            'Avg_Paragraph_Time': format_time_duration(timing_data.get('avg_paragraph_time', 0)),
            'Total_API_Time': format_time_duration(timing_data.get('total_api_time', 0)),
            'API_Time_Ratio_%': round((timing_data.get('total_api_time', 0) / timing_data.get('total_execution_time', 1)) * 100, 1),
            'Questions_Per_Hour': round(3600 / timing_data.get('avg_question_time', 1), 0) if timing_data.get('avg_question_time', 0) > 0 else 0
        })
    
    df = pd.DataFrame(data)
    return df

def analyze_paragraph_timing(timing_results: Dict[str, Dict]) -> pd.DataFrame:
    """단락별 시간 분석"""
    
    all_paragraph_data = []
    
    for combination_key, timing_data in timing_results.items():
        model, prompt = combination_key.split('_', 1)
        paragraph_timings = timing_data.get('paragraph_timings', [])
        
        for p_data in paragraph_timings:
            all_paragraph_data.append({
                'Combination': combination_key,
                'Model': model,
                'Prompt': prompt,
                'Problem_Set_ID': p_data['problem_set_id'],
                'Paragraph_Length': p_data['paragraph_length'],
                'Num_Questions': p_data['num_questions'],
                'Paragraph_Time_s': round(p_data['paragraph_time'], 2),
                'Avg_Question_Time_s': round(p_data['avg_question_time'], 3),
                'Time_Per_Character_ms': round((p_data['paragraph_time'] / p_data['paragraph_length']) * 1000, 2),
                'Questions_Per_Minute': round(60 / p_data['avg_question_time'], 1) if p_data['avg_question_time'] > 0 else 0
            })
    
    df = pd.DataFrame(all_paragraph_data)
    return df

def create_pivot_table(df: pd.DataFrame) -> pd.DataFrame:
    """피벗 테이블 생성 (모델 vs 프롬프트)"""
    pivot = df.pivot(index='Model', columns='Prompt', values='Score')
    return pivot

def create_timing_pivot_table(df: pd.DataFrame) -> pd.DataFrame:
    """시간 관련 피벗 테이블 생성"""
    timing_pivot = df.pivot(index='Model', columns='Prompt', values='Avg_Question_Time_s')
    return timing_pivot


# 결과 테이블 생성
실험 결과를 다양한 관점에서 분석하고 테이블 형태로 정리합니다. 성능과 시간 효율성을 모두 고려한 피벗 테이블과 통계를 생성하여 최적의 모델-프롬프트 조합을 식별할 수 있도록 구성합니다.

In [40]:
results_df = create_results_table(combination_scores, timing_results)
timing_analysis_df = create_timing_analysis_table(timing_results)
paragraph_analysis_df = analyze_paragraph_timing(timing_results)
pivot_df = create_pivot_table(results_df)
timing_pivot_df = create_timing_pivot_table(results_df)

print("\n📊 전체 결과 요약 (성능 + 시간)")
print("=" * 100)
print(results_df.to_string(index=False))

print(f"\n📈 피벗 테이블 (점수)")
print("=" * 50)
print(pivot_df.to_string())

print(f"\n⏱️  시간 분석 상세")
print("=" * 80)
print(timing_analysis_df.to_string(index=False))

print(f"\n📏 평균 문제 처리 시간 (초)")
print("=" * 50)
print(timing_pivot_df.to_string())

# 최고 성능 조합 찾기
best_combination = results_df.loc[results_df['Score'].idxmax()]
fastest_combination = results_df.loc[results_df['Avg_Question_Time_s'].idxmin()]

print(f"\n🏆 최고 성능 조합:")
print(f"   모델: {best_combination['Model']}")
print(f"   프롬프트: {best_combination['Prompt']}")
print(f"   점수: {best_combination['Score']}/{best_combination['Max_Score']}점")
print(f"   정확도: {best_combination['Accuracy_%']}%")
print(f"   문제당 평균 시간: {format_time_duration(best_combination['Avg_Question_Time_s'])}")

print(f"\n⚡ 최고 속도 조합:")
print(f"   모델: {fastest_combination['Model']}")
print(f"   프롬프트: {fastest_combination['Prompt']}")
print(f"   문제당 평균 시간: {format_time_duration(fastest_combination['Avg_Question_Time_s'])}")
print(f"   정확도: {fastest_combination['Accuracy_%']}%")
print(f"   시간당 처리 문제 수: {fastest_combination['Questions_Per_Minute']:.1f}문제/분")


📊 전체 결과 요약 (성능 + 시간)
      Model           Prompt  Score  Max_Score  Accuracy_%  Total_Time_s  Avg_Question_Time_s  Avg_Paragraph_Time_s  Questions_Per_Minute
gpt-4o-mini        zero_shot     70        100        70.0         36.84                0.818                  3.35                  73.3
gpt-4o-mini         negative     70        100        70.0         53.39                1.186                  4.85                  50.6
gpt-4o-mini function_calling     73        100        73.0        105.60                2.346                  9.60                  25.6
     gpt-4o        zero_shot     91        100        91.0         65.56                1.456                  5.96                  41.2
     gpt-4o         negative     87        100        87.0        125.50                2.788                 11.41                  21.5
     gpt-4o function_calling     94        100        94.0        123.33                2.740                 11.21                  21.9

📈 피벗 테이블 (점

# 단락별 상세 시간 분석
단락 길이와 문제 수에 따른 처리 시간 패턴을 분석합니다. 글자당 처리 시간을 계산하여 효율성을 평가하고, 가장 긴 단락과 짧은 단락에서의 성능 차이를 비교 분석합니다.

In [41]:
def analyze_paragraph_performance(paragraph_analysis_df: pd.DataFrame):
    """단락별 성능 분석"""
    
    print(f"\n📚 단락별 처리 시간 분석")
    print("=" * 80)
    
    # 각 조합별 평균 통계
    combination_stats = paragraph_analysis_df.groupby('Combination').agg({
        'Paragraph_Time_s': ['mean', 'std', 'min', 'max'],
        'Avg_Question_Time_s': ['mean', 'std'],
        'Time_Per_Character_ms': ['mean', 'std'],
        'Paragraph_Length': 'mean',
        'Num_Questions': 'mean'
    }).round(3)
    
    print("조합별 단락 처리 통계:")
    print(combination_stats.to_string())
    
    # 가장 긴 단락과 짧은 단락 분석
    longest_paragraph = paragraph_analysis_df.loc[paragraph_analysis_df['Paragraph_Length'].idxmax()]
    shortest_paragraph = paragraph_analysis_df.loc[paragraph_analysis_df['Paragraph_Length'].idxmin()]
    
    print(f"\n📏 단락 길이 분석:")
    print(f"가장 긴 단락: {longest_paragraph['Problem_Set_ID']} ({longest_paragraph['Paragraph_Length']}자)")
    print(f"가장 짧은 단락: {shortest_paragraph['Problem_Set_ID']} ({shortest_paragraph['Paragraph_Length']}자)")
    
    # 효율성 분석 (글자당 처리 시간이 가장 낮은 조합)
    efficiency_analysis = paragraph_analysis_df.groupby('Combination')['Time_Per_Character_ms'].mean().sort_values()
    
    print(f"\n⚡ 효율성 랭킹 (글자당 처리 시간이 낮을수록 효율적):")
    for i, (combination, time_per_char) in enumerate(efficiency_analysis.items(), 1):
        model, prompt = combination.split('_', 1)
        print(f"{i}. {model} + {prompt}: {time_per_char:.2f}ms/글자")

def show_detailed_question_timings(combination_key: str, timing_results: Dict[str, Dict]):
    """특정 조합의 문제별 상세 시간 분석"""
    
    if combination_key not in timing_results:
        print(f"조합 {combination_key}를 찾을 수 없습니다.")
        return
    
    timing_data = timing_results[combination_key]
    question_times = timing_data.get('question_timings', [])
    
    if not question_times:
        print("문제별 시간 데이터가 없습니다.")
        return
    
    print(f"\n🔍 {combination_key} 문제별 시간 분석")
    print("=" * 60)
    
    # 기본 통계
    avg_time = sum(question_times) / len(question_times)
    min_time = min(question_times)
    max_time = max(question_times)
    std_dev = (sum((t - avg_time) ** 2 for t in question_times) / len(question_times)) ** 0.5
    
    print(f"총 문제 수: {len(question_times)}")
    print(f"평균 시간: {format_time_duration(avg_time)}")
    print(f"최소 시간: {format_time_duration(min_time)}")
    print(f"최대 시간: {format_time_duration(max_time)}")
    print(f"표준편차: {format_time_duration(std_dev)}")
    
    # 시간 구간별 분포
    time_ranges = [
        (0, 2, "매우 빠름 (2초 미만)"),
        (2, 5, "빠름 (2-5초)"),
        (5, 10, "보통 (5-10초)"),
        (10, 20, "느림 (10-20초)"),
        (20, float('inf'), "매우 느림 (20초 이상)")
    ]
    
    print(f"\n시간 구간별 분포:")
    for min_t, max_t, label in time_ranges:
        count = sum(1 for t in question_times if min_t <= t < max_t)
        percentage = (count / len(question_times)) * 100
        print(f"  {label}: {count}문제 ({percentage:.1f}%)")

# 단락별 성능 분석 실행
analyze_paragraph_performance(paragraph_analysis_df)

# 최고 성능 조합의 상세 분석
best_combination_key = f"{best_combination['Model']}_{best_combination['Prompt']}"
show_detailed_question_timings(best_combination_key, timing_results)



📚 단락별 처리 시간 분석
조합별 단락 처리 통계:
                             Paragraph_Time_s                     Avg_Question_Time_s        Time_Per_Character_ms         Paragraph_Length Num_Questions
                                         mean    std   min    max                mean    std                  mean     std             mean          mean
Combination                                                                                                                                              
gpt-4o-mini_function_calling            9.600  7.578  4.26  31.31               2.348  1.847                 8.024  10.680           1570.0         4.091
gpt-4o-mini_negative                    4.853  3.183  2.46  11.16               1.264  0.994                 3.675   3.460           1570.0         4.091
gpt-4o-mini_zero_shot                   3.348  1.014  2.15   5.28               0.814  0.117                 2.363   0.817           1570.0         4.091
gpt-4o_function_calling                11.210 

# 시간 효율성 비교 및 권장사항
결과를 바탕으로 다양한 사용 상황별 권장사항을 제시합니다. 정확도 중심, 속도 중심, 균형 중심, 비용 효율성 중심의 네 가지 관점에서 조합을 추천합니다.

In [42]:
def generate_recommendations(results_df: pd.DataFrame, timing_results: Dict[str, Dict]):
    """결과를 바탕으로 권장사항 생성"""
    
    print(f"\n🎯 모델과 프롬프트 조합 비교")
    print("=" * 60)
    
    # 성능 vs 속도 분석
    results_df['Performance_Score'] = results_df['Accuracy_%'] / 100  # 0-1 정규화
    results_df['Speed_Score'] = (1 / results_df['Avg_Question_Time_s']) / (1 / results_df['Avg_Question_Time_s'].max())  # 속도 점수
    results_df['Efficiency_Score'] = (results_df['Performance_Score'] + results_df['Speed_Score']) / 2  # 종합 효율성
    
    # 상황별 좋은 조합
    print("1. 최고 정확도:")
    top_accuracy = results_df.nlargest(2, 'Accuracy_%')
    for idx, row in top_accuracy.iterrows():
        print(f"   - {row['Model']} + {row['Prompt']}: {row['Accuracy_%']}% (문제당 {format_time_duration(row['Avg_Question_Time_s'])})")
    
    print(f"\n2. 빠른 처리:")
    top_speed = results_df.nsmallest(2, 'Avg_Question_Time_s')
    for idx, row in top_speed.iterrows():
        print(f"   - {row['Model']} + {row['Prompt']}: {format_time_duration(row['Avg_Question_Time_s'])}/문제 (정확도 {row['Accuracy_%']}%)")
    
    print(f"\n3. 균형잡힌 성능:")
    top_efficiency = results_df.nlargest(2, 'Efficiency_Score')
    for idx, row in top_efficiency.iterrows():
        efficiency_pct = row['Efficiency_Score'] * 100
        print(f"   - {row['Model']} + {row['Prompt']}: 효율성 {efficiency_pct:.1f}% (정확도 {row['Accuracy_%']}%, 속도 {format_time_duration(row['Avg_Question_Time_s'])})")
    
    # 비용 효율성 분석 (GPT-4o vs GPT-4o-mini)
    print(f"\n4. 비용 대비 성능 분석:")
    mini_results = results_df[results_df['Model'] == 'gpt-4o-mini']
    gpt4_results = results_df[results_df['Model'] == 'gpt-4o']
    
    if not mini_results.empty and not gpt4_results.empty:
        best_mini = mini_results.loc[mini_results['Accuracy_%'].idxmax()]
        best_gpt4 = gpt4_results.loc[gpt4_results['Accuracy_%'].idxmax()]
        
        accuracy_diff = best_gpt4['Accuracy_%'] - best_mini['Accuracy_%']
        time_diff = best_gpt4['Avg_Question_Time_s'] - best_mini['Avg_Question_Time_s']
        
        print(f"   GPT-4o-mini 최고 성능: {best_mini['Accuracy_%']}% ({format_time_duration(best_mini['Avg_Question_Time_s'])}/문제)")
        print(f"   GPT-4o 최고 성능: {best_gpt4['Accuracy_%']}% ({format_time_duration(best_gpt4['Avg_Question_Time_s'])}/문제)")
        print(f"   성능 차이: {accuracy_diff:+.1f}%p, 시간 차이: {format_time_duration(abs(time_diff))}")
        
        if accuracy_diff < 5:  # 5%p 미만 차이
            print("   → 비용 효율성 관점에서 GPT-4o-mini 권장")
        else:
            print("   → 성능 차이를 고려하여 GPT-4o 권장")

generate_recommendations(results_df, timing_results)


🎯 모델과 프롬프트 조합 비교
1. 최고 정확도:
   - gpt-4o + function_calling: 94.0% (문제당 2.74s)
   - gpt-4o + zero_shot: 91.0% (문제당 1.46s)

2. 빠른 처리:
   - gpt-4o-mini + zero_shot: 818ms/문제 (정확도 70.0%)
   - gpt-4o-mini + negative: 1.19s/문제 (정확도 70.0%)

3. 균형잡힌 성능:
   - gpt-4o-mini + zero_shot: 효율성 205.4% (정확도 70.0%, 속도 818ms)
   - gpt-4o-mini + negative: 효율성 152.5% (정확도 70.0%, 속도 1.19s)

4. 비용 대비 성능 분석:
   GPT-4o-mini 최고 성능: 73.0% (2.35s/문제)
   GPT-4o 최고 성능: 94.0% (2.74s/문제)
   성능 차이: +21.0%p, 시간 차이: 394ms
   → 성능 차이를 고려하여 GPT-4o 권장
